In [2]:
## Import
'''
    additionaly required packages (besides the ones in VLNS):
    fiona
    global_land_mask
    wget
    descartes
    scipy
    matplotlib
'''
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import math
from pandas import DataFrame
import geopandas as gpd
import shapely.geometry
import numpy as np
from numpy.random import default_rng
import wget
import shapefile
import plotly.graph_objects as go
from shapely.geometry import mapping, Polygon, Point, MultiPolygon
import fiona
from global_land_mask import globe    # 1km precision
from geopandas import GeoSeries
import random
from shapely.ops import unary_union
rng = default_rng()
from scipy.stats import gamma
from operator import add
from descartes import PolygonPatch

central_europe = gpd.GeoDataFrame.from_file("intersection.shp")
i = np.argmax(central_europe.geometry.area)
europe_polygon = central_europe.geometry[i][0]
europe = europe_polygon
#central_europe.plot() # for visualization

In [3]:
def calc_dist(lat_start, lat_end, lon_start, lon_end):
    # Input: latitude & longitude of two points (example below)
    # Output: (air-line distance in km and) estimated time difference in minutes
    ''' 
    CALCULATION OF DISTANCE IN MINUTES (ROUNDED) OF TWO COORDINATES
    with fixed speed and bee-line distance
    '''
    speed = 75 # estimated avg speed in km/h
    R = 6373.0
    lat2=math.radians(lat_end)
    lat1=math.radians(lat_start)
    lon2=math.radians(lon_end)
    lon1=math.radians(lon_start)
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return  np.ceil(distance/speed*60)

In [4]:
def poly_random(number, polygon_name, dist='uniform'):
    points = []
    minx, miny, maxx, maxy = polygon_name.bounds
    print(minx, miny, maxx, maxy)
    while len(points) < number:
        if dist == 'uniform':
            pnt = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        elif dist == 'normal':
            pnt = Point(np.random.normal(9,5), np.random.normal(51.5,5))
        else: 
            print('Error: Distribution not supported')
            return()
        if polygon_name.contains(pnt):
            points.append(pnt)
    return points

In [5]:
def find_partner2(lon, lat, dist):   # dist in min
    rng = default_rng()
    dist_km = dist/60 * 75
    stopping = 0
    it = 0
    
    while(it < 20):
        degree = rng.integers(0,359)     # 90° for north, 0° for east
        x = np.cos(degree*np.pi/180)     # factors for direction
        y = np.sin(degree*np.pi/180)

        new_lat = lat + y*dist_km/111
        new_lon = lon + x*dist_km/84
        new_dist = calc_dist(lat, new_lat, lon, new_lon)
        diff = dist - new_dist           # neg if we went too far
        old_lat = new_lat
        old_lon = new_lon
        it += 1
        while(abs(diff) > 15)and(europe.contains(Point(new_lon, new_lat))):
            new_lat = old_lat + y*diff/4000*dist_km/111
            new_lon = old_lon + x*diff/4000*dist_km/111
            new_dist = calc_dist(lat, new_lat, lon, new_lon)
            diff = dist - new_dist
            old_lat = new_lat
            old_lon = new_lon
        if((abs(diff) <= 15)and(europe.contains(Point(new_lon, new_lat)))):
            return(new_lon, new_lat, new_dist)
    print('Error, no partner found')
    return(0, 0, 0)

In [6]:
def round_mod(val, floor, mod=15):
    if floor==1:
        return np.floor(val / mod)  *  mod
    else: return np.ceil(val / mod)  *  mod

## TimeWindows

In [7]:
def time_windows(N):
    dist = np.array([0.00172771, 0.07657489, 0.24306138, 0.41039656, 0.5364587,  0.61488255,
                     0.66019702, 0.68662794, 0.71351351, 0.77315484, 0.87388735, 0.96160647, 1.        ])
    rng = default_rng()
    intervals = (rng.poisson(7.7,N)-1)*60                           # intervals in minutes
    intervals[intervals<=0] = rng.integers(1,43)*60                 # noise
    durations = np.array([rng.integers(i-60,i) for i in intervals]) # random minutes in [interval interval-1h] 
    durations[durations<30] = 30                                    # no less than 30min
    regular_begin = 6*60                                            # 6 A.M.
    regular_end = 4*60                                              # 4 hours before midnight = 8 P.M.
    starts = []
    ends = []
    
    for duration in durations:
        if duration < 14*60:
            if duration <= 4*60:
                day = rng.integers(0,5)*24*60
                r = rng.uniform(0,1)
                t_interval = len(dist[dist<r])*60+60*6     # dist contains the probabilities for the ..
                start = rng.integers(t_interval, t_interval+60)+day # intervals 0 to 13 for hours 6 to 20
                starts.append(start) 
                ends.append(start + duration)
            else:  
                day = rng.integers(0,5)*24*60
                start = rng.integers(day+regular_begin, day+24*60-duration-regular_end)
                starts.append(start) 
                ends.append(start + duration)
        else: 
            start = rng.integers(0,8640-duration)         # completely random over the week
            starts.append(start) 
            ends.append(start + duration)   
            
    return(starts, ends)

In [8]:
def P_and_D_windows(starts, ends):
    # starts and ends from delivery windows 
    # Probabilities for P/D Window smaller than 1h, between 1 & 2h and so on:
    pick_probs = np.array([0.397979798,0.309090909,0.084848485,0.008080808,0.02020202,
                           0.01010101,0.018181818,0.042424242,0.046464646,0.016161616,
                           0.016161616,0.002020202,0.006060606,0.004040404,0.018181818])
    del_probs = np.array([0.39061257, 0.28321400, 0.01471758, 0.02665076, 0.04256165, 
                      0.05449483, 0.06642800, 0.06841687, 0.04057279, 0.01073986, 0.00159109])
    pick_cumprob = np.cumsum(pick_probs)
    del_cumprob = np.cumsum(del_probs)
    # time intervals and actual  according to our dist:
    pick_interval = []
    del_interval = [] 
    try:
        N = len(starts)
    except:
        N = 1
    
    for i in range(N):
        r = rng.uniform()
        pick_interval.append(len(pick_cumprob)-len(pick_cumprob[r<=pick_cumprob]))
        r = rng.uniform()
        del_interval.append(len(del_cumprob)-len(del_cumprob[r<=del_cumprob]))
        
    # length in time interval: starting at 30min, 15 min grid
    pick_len = []
    del_len = []  
    for iv in pick_interval:
        r = rng.integers(0,4)
        v = rng.integers(2,4)
        if iv==0:
            pick_len.append(v*15)
        else:
            pick_len.append(iv*60 + r*15)
    for deli in del_interval:
        r = rng.integers(0,4)
        v = rng.integers(2,4)
        if deli==0:
            del_len.append(v*15)
        else:
            del_len.append(deli*60 + r*15)
            
    # pauses & buffer for delivery time window:
    distances = np.array(ends)-np.array(starts)
    k_rest = np.floor(distances/540)
    k_break = np.floor(distances/270) - k_rest
    min_driving_pauses = 660*k_rest +45*k_break
    # buffer for service 90 min (and randomness)
    buffer = 90 #rng.integers(6,12)*15
    
    # pick start = time interval start
    pick_start = starts   
    pick_end = np.array(starts) + np.array(pick_len)
    # no window overlapping for comparibility to other Master Thesis
    delivery_start = pick_end + distances + np.array(min_driving_pauses) + buffer 
    delivery_end = np.array(delivery_start) + np.array(del_len)
    
    return(pick_start, pick_end, delivery_start, delivery_end)

## Finished Data Generation

In [12]:
def data_generation(number_of_requests, Sheetname, printing_i = 10000):
    
    N = number_of_requests
    buffer = int(N/50+5)
    rng = default_rng()
    s,e = time_windows(N+buffer)
    start = np.array(s)
    end = np.array(e)
    planned_durations = end - start
    
    starts = []
    ends = []
    end_lats =[]
    planned_dist = []
    end_lons = []
    start_lats=[]
    start_lons=[]
    points_N = poly_random(N+buffer, europe,'normal')
    real_distance = []
    
    i=0
    k=0
    for p in points_N:
        lon, lat = p.coords[0]
        dist = planned_durations[i]
        lon1, lat1, diff = find_partner2(lon, lat, dist)
        if lon1!=0:
            planned_dist.append(dist)
            end_lons.append(lon1)
            end_lats.append(lat1)
            start_lats.append(lat)
            start_lons.append(lon)
            real_distance.append(diff)
            starts.append(start[i])
            ends.append(end[i])
            k+=1
        if k == N:
            break
        i+=1
        if i%printing_i==0:
            print('iter: ',i)
    
    up_or_down = rng.integers(0,2,k)   #rounds randomly start or end down/up
    differences = np.array(real_distance) - np.array(planned_dist)
    mods = differences/2        
    diff1 = up_or_down *  np.ceil(mods) +(1-up_or_down)* np.floor(mods)
    diff2 = up_or_down * np.floor(mods) +(1-up_or_down)* np.ceil(mods)
    s_new = np.array(starts) - diff1
    e_new = np.array(ends) + diff2
    
    # catch s<0 and e>'6 days' (unmöglich beides):
    e_new[s_new<0]    -= s_new[s_new<0]
    s_new[s_new<0]     = 0
    s_new[e_new>8640]-= e_new[e_new>8640]
    e_new[e_new>8640] = 8640
    
    #print(starts,ends,'\n',s_new,e_new)
    pick_start, pick_end, delivery_start, delivery_end = P_and_D_windows(s_new, e_new)
    
    print('valid points: ',k)
    Data = DataFrame({'lat1': start_lats, 'lon1': start_lons,'lat2': end_lats, 'lon2': end_lons, 'pick_start': pick_start, 
                      'pick_end': pick_end, 'delivery_start': delivery_start, 'delivery_end': delivery_end})
    
    with pd.ExcelWriter(Sheetname, engine='openpyxl', mode='a') as writer: #mode a = neues Blatt !!!
         Data.to_excel(writer, sheet_name="Data_N_"+str(N), index=False)
    return()

In [17]:
## Generating k instances with n requests 
# and printing an update every p steps:
k = 20
n = 100
p = 50
for i in range(k):
    data_generation(n, 'Test.xlsx', p)

-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
Error, no partner found
Error, no partner found
iter:  100
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
Error, no partner found
Error, no partner found
iter:  100
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
-7.4 39.0 20.0 55.0
Error, no partner found
iter:  100
valid points:  100
-7.4 39.0 20.0 55.0
valid points:  100
